## 本分类问题有很多经典解决办法

- 朴素贝叶斯分类器（Naive Bayes Classifier）通过某些条件发生的概率来进行分类。
- 支持向量机（Support Vector Machine, SVM）通过寻找最佳分割超平面来进行分类。
- 决策树（Decision Tree）通过树状结构进行分类。
- 随机森林（Random Forest）通过集成多个决策树进行分类。
- LDA（Latent Dirichlet Allocation）通过主题建模进行分类。
- 神经网络（Neural Networks）通过多层感知器进行分类。

面临主要问题
1. 类别多样性：情感类别多样，可能存在多个情感标签，增加分类难度。
2. 语言复杂性：自然语言表达复杂，包含隐含情感、讽刺、双关等，难以准确捕捉情感倾向。
3. 数据不平衡：某些情感类别的数据量较少，导致模型难以学习这些类别的特征。
4. 上下文依赖性：情感表达往往依赖上下文，单独的句子可能无法准确反映情感倾向。
5. 多义词和同义词：词语可能具有多重含义或相似含义，增加情感分类的复杂性。
6. 多语言支持：不同语言的情感表达方式不同，增加跨语言情感分类的难度。

## BERT
BERT 采用了基于 MLM 的模型训练方式，即 Mask Language Model。因为 BERT 是 Transformer 的一部分，即 encoder 环节，所以没有 decoder 的部分（其实就是 GPT）。

为了解决这个问题，MLM 方式应运而生。它的思想也非常简单，就是在训练之前，随机将文本中一部分的词语（token）进行屏蔽（mask），然后在训练的过程中，使用其他没有被屏蔽的 token 对被屏蔽的 token 进行预测。

``` shell
wget https://storage.googleapis.com/bert_models/2018_11_23/multi_cased_L-12_H-768_A-12.zip
unzip multi_cased_L-12_H-768_A-12.zip
```
### 格式准备

- Token embeddings：词向量。这里需要注意的是，Token embeddings 的第一个开头的 token 一定得是“[CLS]”。[CLS]作为整篇文本的语义表示，用于文本分类等任务。
- Segment embeddings。这个向量主要是用来将两句话进行区分，比如问答任务，会有问句和答句同时输入，这就需要一个能够区分两句话的操作。不过在咱们此次的分类任务中，只有一个句子。
- Position embeddings。位置向量。Transformer 结构中没有 RNN 那样的时序信息，所以需要通过位置向量来表示词语在句子中的位置。

### 模型配置

id2label：这个字段记录了类别标签和类别名称的映射关系。

label2id：这个字段记录了类别名称和类别标签的映射关系。

num_labels_cate：类别的数量。数据准备

1. InputExample：它用于记录单个训练数据的文本内容的结构。
2. DataProcessor：通过这个类中的函数，我们可以将训练数据集的文本，表示为多个 InputExample 组成的数据集合。
3. get_features：用于把 InputExample 数据转换成 BERT 能够理解的数据结构的关键函数。我们具体来看一下各个数据都怎么生成的。

input_ids 记录了输入 token 对应在 vocab.txt 的 id 序号，它是通过如下的代码得到的。

In [1]:
# 环境与依赖（可重复执行）
%pip install -q transformers datasets accelerate scikit-learn

import torch, transformers, datasets, sklearn
print("Versions:")
print("  torch:", torch.__version__)
print("  transformers:", transformers.__version__)
print("  datasets:", datasets.__version__)


Note: you may need to restart the kernel to use updated packages.


/home/uceeqz4/.conda/envs/pytorch311/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Versions:
  torch: 2.5.1+cu121
  transformers: 4.57.1
  datasets: 4.4.0


## Bert 模型数据以及训练（IMDB 全流程详解）

下面详细说明从 IMDB 原始数据到可训练张量的“数据流”，以及训练的每个关键细节与超参数设置。配套代码见当前 Notebook 中的相邻代码单元（数据加载/特征化/训练）。

### 1) 数据集与划分
- 使用 Hugging Face Datasets 的 `imdb` 数据集：包含 `train` 与 `test` 两个官方子集（各 25k 条）；`train` 内再划出验证集。
- 本笔记采用：
  - 训练集：`train[:95%]`
  - 验证集：`train[95%:]`
  - 测试集：`test`
- 在代码中，先将每条样本包装成 `InputExample(guid, text_a, label)`，便于后续统一处理。

### 2) 分词器与特征化（Tokenizer → 模型可用特征）
- 分词器：`AutoTokenizer.from_pretrained(MODEL_NAME)`，这里 `MODEL_NAME = "bert-base-uncased"`。
- 关键参数：
  - `truncation=True`：超过 `max_length` 的序列会被截断；本例 `max_length=256`。
  - `padding=False`（在单样本阶段不做 padding），padding 交给批处理函数 `collate_batch` 动态完成。
  - `return_tensors="pt"`：直接返回 PyTorch 张量。
- 自定义 `Dataset`（`BertTextDataset`）的 `__getitem__`：
  - 取出第 `idx` 条 `InputExample`，调用 `_tokenizer(text_a, text_b, ...)` 得到字典：`{"input_ids", "token_type_ids"(可能), "attention_mask"}`，形状均为 `[L]`。
  - 若存在标签：附加 `{"labels": LongTensor(0/1)}`。
  - 返回单样本的特征字典（张量）。

#### 2.1) input_ids 与 attention_mask 的来源与原理
- input_ids：
  - 文本先被拆分为词片（WordPiece/BPE），再映射到词表中的整数 ID。
  - BERT 单句输入的典型形式为：`[CLS] tokens ... [SEP]`；若是句对（text_a, text_b），则为：`[CLS] tokens_a [SEP] tokens_b [SEP]`。
  - 例如（示意，不代表真实 ID）：
    - 文本："I love this movie!" → 词片：`[CLS, i, love, this, movie, !, SEP]`
    - input_ids：`[101, 1045, 2293, 2023, 3185, 999, 102]`（ID 取决于词表）。
- attention_mask：
  - 由分词器在 padding/截断后自动生成，规则是“真实 token（含特殊符号 CLS/SEP）为 1，padding 出来的占位为 0”。
  - 在单样本阶段我们设置 `padding=False`，attention_mask 仅体现真实 token；在批处理阶段通过 `_tokenizer.pad(...)` 进行动态填充后，mask 会相应扩展为 `[B, T]`：
    - 真实 token → 1
    - PAD 位置 → 0
  - 例如批大小 B=3，按批内最长序列 T=8 填充后，一个样本可能为：
    - input_ids: `[101, 1045, 2293, 2023, 3185, 999, 102, 0]`
    - attention_mask: `[1,    1,    1,    1,    1,   1,   1, 0]`
- 模型内部如何使用 attention_mask：
  - 在自注意力（Self-Attention）里，模型会把 mask 为 0 的位置对应的 key（以及通常也包括 query 的输出）屏蔽（通过在注意力分数上加上一个大负数，使 softmax 后权重近似为 0），从而确保 PAD 不参与信息交互，也不干扰有效 token 的表示。
  - 对于句对任务，`token_type_ids`（Segment IDs）可区分 A 句与 B 句（A=0, B=1）。本例为单句分类，通常全 0；很多现代模型也不再使用该特征。
- 截断与填充策略：
  - 截断默认 `longest_first`（若是句对会优先从较长一侧裁剪），确保总长度不超过 `max_length`。
  - 填充使用“批内动态填充”，只 pad 到当前批次的最长序列（或到 `max_length`，取决于参数），能更高效地利用算力与显存。

### 3) 批处理与动态填充（Collate → 等长批次）
- `collate_batch(features)`：
  - 调用 `_tokenizer.pad(features, padding=True, max_length=max_length, return_tensors="pt")`。
  - 将不同长度的样本在批内按同一最大长度（或 `max_length`）做 padding，得到：
    - `input_ids`: `[B, T]`, `attention_mask`: `[B, T]`,（若存在）`token_type_ids`: `[B, T]`。
    - `labels`: `[B]`。
- 这样，模型前向时可以一次性处理一个批次，显著提升吞吐。

### 4) DataLoader（高效喂数）
- 训练/验证/测试分别构建 `DataLoader`：
  - 训练：`batch_size=16, shuffle=True`
  - 验证/测试：`batch_size=32, shuffle=False`
- 为了提高 GPU 利用率，训练单元中使用了：
  - `num_workers` > 0（多进程加载），`pin_memory=True`（加速 H2D 拷贝），`prefetch_factor` + `persistent_workers=True`（减少加载抖动）。
  - 张量迁移到 GPU 时使用 `non_blocking=True`。

### 5) 模型与损失（BertForSequenceClassification）
- 模型：`BertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)`。
- 输入：`input_ids, attention_mask, (可选) token_type_ids, labels`；输出 `SequenceClassifierOutput`：
  - `loss`: 标量（当提供 `labels` 时自动计算交叉熵）
  - `logits`: `[B, 2]`（二分类）
- 精度与加速：
  - AMP 混合精度（优先 bfloat16，否则 float16），并启用 TF32（若硬件支持）。

### 6) 优化与调度
- 优化器：`AdamW(lr=2e-5)`。
- 训练轮数：`epochs=3`（可根据显存与目标精度调整）。
- 学习率调度：线性 warmup + 线性衰减，`warmup_steps = 10% * total_steps`。
- `optimizer.zero_grad(set_to_none=True)` 降低内存碎片。

### 7) 训练-验证-测试循环（度量与保存）
- 每个 epoch：
  - Train 阶段：前向得到 `loss` → 反向传播（AMP 下用 `GradScaler`）→ `optimizer.step()` → `scheduler.step()`。
  - 记录 `train_loss / train_acc`（`acc` 由 `logits.argmax(-1)` 与 `labels` 对比得到）。
  - Validate 阶段：`model.eval()` + `torch.no_grad()`，计算 `val_loss / val_acc`。
  - Early best 保存：当 `val_loss` 创新低时，`model.save_pretrained(ckpt_dir2)` 与 `_tokenizer.save_pretrained(ckpt_dir2)`。
- 训练结束：在测试集 `test_loader` 上报告 `test_loss / test_acc`。

### 8) 张量形状与“数据流”一览
- 单样本（getitem 后）：
  - `input_ids`: `[L]`（Long），`attention_mask`: `[L]`（Long），（可选）`token_type_ids`: `[L]`（Long），`labels`: `[]`（标量 Long）
- 批处理（collate 后）：
  - `input_ids`: `[B, T]`（Long），`attention_mask`: `[B, T]`（Long），`labels`: `[B]`（Long）
- 前向输出：
  - `logits`: `[B, 2]`（Float16/BFloat16/Float32）
- 全链路数据流：
  1. 原始文本 → `InputExample`（含 `text_a`, `label`）
  2. `Dataset.__getitem__` → 调用 `tokenizer(...)` → 单样本特征字典（张量）
  3. `collate_batch` → 动态 padding 成等长批次
  4. `DataLoader` → 按批输出到训练循环
  5. GPU 上 `model(**batch)` → `loss` + `logits`
  6. 反向传播与优化 → 验证 → 选择最优权重并保存

### 9) 常见可调项与建议
- 更快的数据侧：可将“逐样本分词”替换为一次性 `datasets.map` 预分词，并 `set_format(type='torch')`，进一步减轻 DataLoader 压力。
- 提高利用率：适当增大 `batch_size`（或用梯度累积），确保 `num_workers/pin_memory` 设置合理。
- 复现/部署：用 `save_pretrained` 持久化模型与分词器；推理时 `from_pretrained(ckpt_dir2)` 即可加载。

### 10) 优化器参数分组与权重衰减（no_decay 策略）
- 目的：对不同参数施加不同的 weight decay，一般做法是“对偏置（bias）与 LayerNorm 权重不做衰减，其余权重做 L2 衰减”，经验上更稳定、更有利于泛化。
- 典型实现：

```python
# 参数分组示例（排除 bias 和 LayerNorm 权重的权重衰减）
param_optimizer = list(model.named_parameters())
no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
optimizer_grouped_parameters = [
    {
        "params": [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
        "weight_decay": 0.01,
    },
    {
        "params": [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },
]
optimizer = torch.optim.AdamW(
    optimizer_grouped_parameters,
    lr=2e-5,
    eps=1e-8,  # 等价于 transformers.AdamW(..., eps=args.adam_epsilon)
)
```

- 与本笔记当前默认设置的区别：
  - 本笔记训练代码中使用 `torch.optim.AdamW(model.parameters(), lr=lr)`，未显式设置 `weight_decay`，默认等于 0，对所有参数“不做权重衰减”。
  - 上述分组写法对“除 bias/LayerNorm 外的参数”施加 `weight_decay=0.01`，对 `bias/LayerNorm` 保持 `0.0`，效果等价于 “L2 只作用在权重矩阵，不作用在偏置与归一化参数”。
- 何时推荐：
  - 微调 BERT/Transformer 家族模型时是业界常用默认；若出现过拟合、验证集指标不稳，可优先尝试该策略。
- 替换方式：
  - 若要在本训练循环中启用，只需将创建 `optimizer` 的那一行替换为上述分组写法；其余训练/调度代码保持不变。


In [2]:
# InputExample 与 Processor 定义（IMDB 二分类示例）
from dataclasses import dataclass
from typing import Optional, List, Dict
from datasets import load_dataset

@dataclass
class InputExample:
    guid: str
    text_a: str
    text_b: Optional[str] = None
    label: Optional[int] = None  # 0/1


def load_imdb_as_examples(split: str) -> List[InputExample]:
    ds = load_dataset("imdb", split=split)
    examples = []
    for i, ex in enumerate(ds):
        guid = f"imdb-{split}-{i}"
        examples.append(InputExample(guid=guid, text_a=ex["text"], label=int(ex["label"])))
    return examples

# 演示：取前2个样本看一下结构
train_examples_preview = load_imdb_as_examples("train[:2]")
train_examples_preview


[InputExample(guid='imdb-train[:2]-0', text_a='I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex

In [ ]:
# Tokenizer、特征化与小批量数据构造
from transformers import AutoTokenizer
from torch.utils.data import Dataset
import torch

MODEL_NAME = "bert-base-uncased"
max_length = 256

_tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

class BertTextDataset(Dataset):
    def __init__(self, examples: List[InputExample]):
        self.examples = examples
    def __len__(self):
        return len(self.examples)
    def __getitem__(self, idx):
        ex = self.examples[idx]
        # truncation=True, max_length=256, padding=False，所以只“截断不补齐”，每个样本长度 ≤ 256。
        enc = _tokenizer(
            ex.text_a,
            ex.text_b,
            truncation=True,
            padding=False,
            max_length=max_length,
            return_tensors="pt",
        )
        item = {k: v.squeeze(0) for k, v in enc.items()}  # remove batch dim
        if ex.label is not None:
            item["labels"] = torch.tensor(ex.label, dtype=torch.long)
        return item

from torch.utils.data import DataLoader

def collate_batch(features: List[Dict[str, torch.Tensor]]):
    # padding=True 做的是“批内动态填充”，会 pad 到本批次中“最长样本的长度”，因此实际批长 T ≤ 256；只有当这一批里恰好存在长度=256 的样本时，T 才会等于 256。
    return _tokenizer.pad(
        features,
        padding=True,
        max_length=max_length,
        return_tensors="pt",
    )

# 构建 train/validation/test 划分
train_examples = load_imdb_as_examples("train[:95%]")
valid_examples = load_imdb_as_examples("train[95%:]")
test_examples  = load_imdb_as_examples("test")

train_ds = BertTextDataset(train_examples)
valid_ds = BertTextDataset(valid_examples)
test_ds  = BertTextDataset(test_examples)

len(train_ds), len(valid_ds), len(test_ds)

# 测试一个batch 包含的内容
# 取一个小批次并打印关键字段与形状/类型；同时示例解码第一条样本
_demo_loader = DataLoader(train_ds, batch_size=4, shuffle=False, collate_fn=collate_batch)
_batch = next(iter(_demo_loader))
print("batch keys:", list(_batch.keys()))
for k, v in _batch.items():
    print(f"  {k:>14}: shape={tuple(v.shape)}, dtype={v.dtype}")

if "token_type_ids" in _batch:
    print("token_type_ids 存在，示例前 16 位:", _batch["token_type_ids"][0][:16].tolist())

if "labels" in _batch:
    print("labels:", _batch["labels"].tolist())

# 展示首条样本的部分 input_ids / attention_mask，并做一次可读解码
print("sample[0] input_ids (前 30):", _batch["input_ids"][0][:30].tolist())
print("sample[0] attention_mask (前 30):", _batch["attention_mask"][0][:30].tolist())
try:
    print("decoded[0] (前 200 字符):", _tokenizer.decode(_batch["input_ids"][0], skip_special_tokens=True)[:200])
except Exception as e:
    print("decode failed:", e)



You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


batch keys: ['input_ids', 'token_type_ids', 'attention_mask', 'labels']
       input_ids: shape=(4, 256), dtype=torch.int64
  token_type_ids: shape=(4, 256), dtype=torch.int64
  attention_mask: shape=(4, 256), dtype=torch.int64
          labels: shape=(4,), dtype=torch.int64
token_type_ids 存在，示例前 16 位: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
labels: [0, 0, 0, 0]
sample[0] input_ids (前 30): [101, 1045, 12524, 1045, 2572, 8025, 1011, 3756, 2013, 2026, 2678, 3573, 2138, 1997, 2035, 1996, 6704, 2008, 5129, 2009, 2043, 2009, 2001, 2034, 2207, 1999, 3476, 1012, 1045, 2036]
sample[0] attention_mask (前 30): [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
decoded[0] (前 200 字符): i rented i am curious - yellow from my video store because of all the controversy that surrounded it when it was first released in 1967. i also heard that at first it was seized by u. s. customs if it


/home/uceeqz4/.conda/envs/pytorch311/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:2779: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


### 关于 collate_batch：为什么 batch 里有 input_ids / attention_mask / token_type_ids / labels？它们是在哪定义的

- collate_batch 接收的是一个列表：`features: List[Dict[str, Tensor]]`，这个列表里的每个字典，来自 Dataset 的 `__getitem__` 返回值聚合。
- 这些键最初“定义”的位置：
  - input_ids、attention_mask、(可选) token_type_ids：由分词器 `_tokenizer(...)` 返回，字段名是 Hugging Face 统一规范（`AutoTokenizer.__call__` 的输出）。
  - labels：在我们的 `__getitem__` 里手动添加的：`item["labels"] = torch.tensor(ex.label, dtype=torch.long)`。
- collate_batch 本身并不“创建字段”，而是调用：
  - `_tokenizer.pad(features, padding=True, max_length=max_length, return_tensors="pt")`
  - 该函数会：
    - 对序列字段（如 `input_ids`、`attention_mask`、`token_type_ids`）按批次做 padding 到同一长度，并生成/扩展 `attention_mask`（真实 token=1，PAD=0）。
    - 对非序列字段（如 `labels`）做堆叠（stack），得到形如 `[B]` 的张量。
- 因此，`batch` 里“有哪些键”，完全取决于单样本 `item` 里“有哪些键”：
  - 如果分词器没返回 `token_type_ids`（很多模型不需要），那 `batch` 里自然也不会有该键。
  - 如果你不想包含某个键，可以在 `__getitem__` 中不添加；想新增自定义特征，也可在 `__getitem__` 中增加字段，`pad` 会尽量按类型合并（非序列字段通常直接 stack）。


In [6]:
# 使用 BertForSequenceClassification 的训练代码（纯 PyTorch 训练循环，含 AMP、DataLoader 优化）
import os, math
import torch
from torch.utils.data import DataLoader
from transformers import BertForSequenceClassification, get_linear_schedule_with_warmup

# 目录（Notebook 中使用工作目录）
ckpt_dir2 = os.path.join(os.getcwd(), "ckpts", "bert_emotion_bertcls")
os.makedirs(ckpt_dir2, exist_ok=True)

# 设备与加速选项
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# 允许 TF32（Ampere+ GPU 有效），可提升吞吐
try:
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.allow_tf32 = True
    torch.set_float32_matmul_precision("high")
except Exception:
    pass

# AMP 设置（优先使用 bfloat16，否则使用 float16）
use_autocast = torch.cuda.is_available()
try:
    autocast_dtype = torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16
except Exception:
    autocast_dtype = torch.float16
scaler = torch.cuda.amp.GradScaler(enabled=(autocast_dtype == torch.float16 and torch.cuda.is_available()))

# DataLoader（多进程、固定内存、预取）
num_workers = max(1, min(8, (os.cpu_count() or 2) // 2))
train_loader = DataLoader(
    train_ds, batch_size=16, shuffle=True, collate_fn=collate_batch,
    num_workers=num_workers, pin_memory=True, persistent_workers=True, prefetch_factor=2
)
valid_loader = DataLoader(
    valid_ds, batch_size=32, shuffle=False, collate_fn=collate_batch,
    num_workers=num_workers, pin_memory=True, persistent_workers=True, prefetch_factor=2
)
test_loader  = DataLoader(
    test_ds,  batch_size=32, shuffle=False, collate_fn=collate_batch,
    num_workers=num_workers, pin_memory=True, persistent_workers=True, prefetch_factor=2
)
print(f"Using device: {device}, num_workers={num_workers}, amp_dtype={autocast_dtype}")

# 创建模型（加载预训练 BERT）
model = BertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)
model.to(device)

# 优化器与调度器
lr = 2e-5
epochs = 3
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
num_training_steps = epochs * len(train_loader)
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=int(0.1 * num_training_steps),
    num_training_steps=num_training_steps,
)

# 简单准确率
def accuracy_from_logits(logits, labels):
    preds = logits.argmax(dim=-1)
    return (preds == labels).float().mean().item()

best_val_loss = math.inf
print("Start training (BertForSequenceClassification)...")
for epoch in range(1, epochs + 1):
    # Train
    model.train()
    tr_loss, tr_acc = 0.0, 0.0
    for batch in train_loader:
        batch = {k: v.to(device, non_blocking=True) for k, v in batch.items()}
        optimizer.zero_grad(set_to_none=True)
        with torch.cuda.amp.autocast(enabled=use_autocast, dtype=autocast_dtype):
            out = model(**batch)  # returns SequenceClassifierOutput
            loss = out.loss
        if scaler.is_enabled():
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            loss.backward()
            optimizer.step()
        scheduler.step()
        tr_loss += loss.item()
        tr_acc  += accuracy_from_logits(out.logits, batch["labels"])    
    tr_loss /= len(train_loader)
    tr_acc  /= len(train_loader)

    # Validate
    model.eval()
    va_loss, va_acc = 0.0, 0.0
    with torch.no_grad():
        for batch in valid_loader:
            batch = {k: v.to(device, non_blocking=True) for k, v in batch.items()}
            with torch.cuda.amp.autocast(enabled=use_autocast, dtype=autocast_dtype):
                out = model(**batch)
                va_loss += out.loss.item()
                va_acc  += accuracy_from_logits(out.logits, batch["labels"])
    va_loss /= len(valid_loader)
    va_acc  /= len(valid_loader)

    # 保存最优
    if va_loss < best_val_loss:
        best_val_loss = va_loss
        model.save_pretrained(ckpt_dir2)
        _tokenizer.save_pretrained(ckpt_dir2)

    print(f"[Epoch {epoch}/{epochs}] train_loss={tr_loss:.4f} train_acc={tr_acc:.4f} | val_loss={va_loss:.4f} val_acc={va_acc:.4f}")

# 测试
model.eval()
te_loss, te_acc = 0.0, 0.0
with torch.no_grad():
    for batch in test_loader:
        batch = {k: v.to(device, non_blocking=True) for k, v in batch.items()}
        with torch.cuda.amp.autocast(enabled=use_autocast, dtype=autocast_dtype):
            out = model(**batch)
            te_loss += out.loss.item()
            te_acc  += accuracy_from_logits(out.logits, batch["labels"])  
te_loss /= len(test_loader)
te_acc  /= len(test_loader)
print(f"[Test] loss={te_loss:.4f} acc={te_acc:.4f}")

print("Saved best model to:", ckpt_dir2)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Start training (BertForSequenceClassification)...
[Epoch 1/3] train_loss=0.3022 train_acc=0.8705 | val_loss=0.1987 val_acc=0.9242
[Epoch 1/3] train_loss=0.3022 train_acc=0.8705 | val_loss=0.1987 val_acc=0.9242
[Epoch 2/3] train_loss=0.1433 train_acc=0.9489 | val_loss=0.1346 val_acc=0.9555
[Epoch 2/3] train_loss=0.1433 train_acc=0.9489 | val_loss=0.1346 val_acc=0.9555
[Epoch 3/3] train_loss=0.0615 train_acc=0.9811 | val_loss=0.2592 val_acc=0.9148
[Epoch 3/3] train_loss=0.0615 train_acc=0.9811 | val_loss=0.2592 val_acc=0.9148
[Test] loss=0.2426 acc=0.9230
Saved best model to: /home/uceeqz4/Project/learning/PyTorch_study/Application/ckpts/bert_emotion_bertcls
[Test] loss=0.2426 acc=0.9230
Saved best model to: /home/uceeqz4/Project/learning/PyTorch_study/Application/ckpts/bert_emotion_bertcls


## model(**batch) 与显式传参的区别，以及 batch 包含哪些内容（示例与说明）

- 两种调用方式
  - 关键字解包：`outputs = model(**batch)` 等价于 `model(input_ids=batch["input_ids"], attention_mask=batch["attention_mask"], token_type_ids=batch.get("token_type_ids"), labels=batch.get("labels"))`。
  - 显式传参：逐个把张量按参数名传入。
- 推荐原因
  - 通用稳健：不同模型对 `token_type_ids` 需求不同（如 RoBERTa/DistilBERT 不需要）。用 `**batch` 时若该键不存在，就不会传入，避免多余参数或告警。
  - 与分词/批处理天然对齐：`tokenizer.pad(...)` 返回的就是“键→张量”的字典，直接喂给模型最顺手。
  - 可读性/安全：不依赖位置参数顺序，减少误用风险。
- 返回值形态（易混点）
  - 现代 Transformers 默认 `return_dict=True`，返回 `SequenceClassifierOutput`，需要用 `outputs.loss`、`outputs.logits` 访问。
  - 只有显式设置 `return_dict=False` 才会返回 tuple（此时才会 `loss, logits = outputs[:2]`）。
- batch 里到底有什么（来自 `collate_batch(tokenizer.pad(...))`）
  - `input_ids` `[B, T]` Long：词表 ID，含 `[CLS]/[SEP]`。
  - `attention_mask` `[B, T]` Long：真实 token 为 1，padding 为 0。
  - `token_type_ids` `[B, T]` Long（可选）：句对任务区分 A/B 段；单句通常全 0；很多模型不使用。
  - `labels` `[B]` Long（可选）：分类标签；提供后模型自动计算 `loss`。


In [ ]:
# 演示：检查 batch 内容与两种调用方式是否等价
import torch

# 设备
_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 准备一个小批次（若 train_loader 尚未构建，则用临时 DataLoader）
try:
    _ = train_loader  # noqa
    demo_loader = train_loader
except NameError:
    from torch.utils.data import DataLoader
    demo_loader = DataLoader(train_ds, batch_size=4, shuffle=False, collate_fn=collate_batch)

batch = next(iter(demo_loader))
print("batch keys:", list(batch.keys()))
for k, v in batch.items():
    print(f"  {k:>14}: shape={tuple(v.shape)}, dtype={v.dtype}")

# 准备模型（若未构建）
try:
    _ = model  # noqa
except NameError:
    from transformers import BertForSequenceClassification
    model = BertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2).to(_device)

model.eval()
with torch.no_grad():
    batch_dev = {k: v.to(_device) for k, v in batch.items()}
    # 方式一：**batch
    out1 = model(**batch_dev)
    # 方式二：显式关键字传参（token_type_ids 可能不存在，用 get）
    out2 = model(
        input_ids=batch_dev["input_ids"],
        attention_mask=batch_dev["attention_mask"],
        token_type_ids=batch_dev.get("token_type_ids", None),
        labels=batch_dev.get("labels", None),
    )

print("outputs1 type:", type(out1).__name__, ", logits shape:", tuple(out1.logits.shape))
print("outputs2 type:", type(out2).__name__, ", logits shape:", tuple(out2.logits.shape))

# 验证两种方式的 logits 一致（数值上应相同）
print("logits allclose:", torch.allclose(out1.logits, out2.logits, atol=1e-6))

if hasattr(out1, "loss") and out1.loss is not None:
    print("loss1:", float(out1.loss))
if hasattr(out2, "loss") and out2.loss is not None:
    print("loss2:", float(out2.loss))


## 结果

比 LSTM 准确很多！！！